# Risk-Reward Analysis (CRV)

This notebook calculates the Risk-Reward Ratio (CRV) for a stock using historical 
EV/EBITDA and PE Ratio data. 

It also provides probability-adjusted CRV using histograms of past ratios.


In [11]:
from src.setup import *

## Methodology

The CRV (Risk-Reward Ratio) is calculated as:

- **EV/EBITDA CRV**: Uses historical EV/EBITDA multiples to estimate upside (Buy → Sell) and downside (Worst Case → Buy) prices.
- **PE Ratio CRV**: Uses historical PE multiples in the same way.
- **Probability-Adjusted CRV**: Adjusts the CRV based on the historical frequency of the current multiple.


In [12]:
# Function Definitions
# 1. calculate_crv() 
# 2. ev_histogram() 
# 3. pe_histogram() 
# 4. chance_adjusted_ev()
# 5. chance_adjusted_pe()


## `calculate_crv()`

Calculates the basic Risk-Reward Ratio (CRV) for a stock using:

- **EV/EBITDA multiples**: determines worst-case, buy, fair-value, and sell levels.
- **PE Ratio multiples**: same logic as above.
- Outputs prices, ratios, and the CRV (chance/risk).
- Handles cases where either EV/EBITDA or PE data is missing.


In [13]:

def calculate_crv(price, ev_ebitda_list, pe_ratio_list, ebitda, net_debt, eps, shares_outstanding, has_ev=False, has_pe=False):
    """
    Calculate the Risk-Reward Ratio (CRV) based on EV/EBITDA and PE Ratio.

    Parameters
    ----------
    price : float
        Current stock price.
    ev_ebitda_list : list of float
        Historical yearly EV/EBITDA values, sorted from smallest to largest.
    pe_ratio_list : list of float
        Historical yearly PE Ratio values, sorted from smallest to largest.
    ebitda : float
        Current EBITDA of the company.
    net_debt : float
        Net debt of the company (total debt minus cash).
    eps : float
        Earnings per share.
    shares_outstanding : float
        Number of shares outstanding.
    has_ev : bool, default False
        Whether EV/EBITDA data is available.
    has_pe : bool, default False
        Whether PE Ratio data is available.

    Returns
    -------
    dict
        Dictionary containing:
        - pe_values, pe_prices : DataFrames with PE levels and prices
        - ev_values, ev_prices : DataFrames with EV/EBITDA levels and prices
        - df_result, df_prices : combined DataFrames
        - Crv_pe, Crv_ev : calculated Risk-Reward Ratios
        - Chance_ev, Risk_ev, Chance_pe, Risk_pe : upside/downside values
    """

    if not has_pe and not has_ev:
        raise ValueError("Neither EV/EBITDA nor PE Ratio data is available.")

    # Initialize outputs
    pe_values = pe_prices = ev_values = ev_prices = None
    Crv_pe = Crv_ev = None
    Chance_pe = Risk_pe = Chance_ev = Risk_ev = None

    # ---- EV/EBITDA ----
    if has_ev:

        # EV/EBITDA reference values based on sorted historical data
        WC_Ev = round((ev_ebitda_list[0] + np.mean(ev_ebitda_list[1:4])) / 2)  # "Worst Case": minimum + lowest 2-4 values average
        Buy_EV = np.mean(ev_ebitda_list[1:4])  # Buy level: average of lowest 2-4 historical values
        Sell_EV = np.mean(ev_ebitda_list[-4:-1])  # Sell level: average of 2-4 highest historical values
        FV_Ev = (Buy_EV + Sell_EV) / 2  # Fair Value: midpoint between Buy and Sell levels


        WC_EV_Price = (ebitda * WC_Ev - net_debt) / shares_outstanding
        Buy_EV_Price = (ebitda * Buy_EV - net_debt) / shares_outstanding
        FV_EV_Price = (ebitda * FV_Ev - net_debt) / shares_outstanding
        Sell_EV_Price = (ebitda * Sell_EV - net_debt) / shares_outstanding

        ev_values = pd.DataFrame({
            "EV/EBITDA": [WC_Ev, Buy_EV, FV_Ev, Sell_EV]
        }, index=["WC", "Buy", "FV", "Sell"])

        ev_prices = pd.DataFrame({
            "EV/EBITDA": [WC_EV_Price, Buy_EV_Price, FV_EV_Price, Sell_EV_Price]
        }, index=["WC", "Buy", "FV", "Sell"])

        if WC_EV_Price < price and Sell_EV_Price < price:
            Chance_ev = 0
            Risk_ev = price - WC_EV_Price
            Crv_ev = 0
        elif WC_EV_Price > price and Sell_EV_Price > price:
            Chance_ev = Sell_EV_Price - price
            Risk_ev = 0
            Crv_ev = float('inf')
        else:
            Chance_ev = Sell_EV_Price - price
            Risk_ev = price - WC_EV_Price
            Crv_ev = Chance_ev / Risk_ev if Risk_ev != 0 else float('inf')

    # ---- PE Ratio ----
    if has_pe:
        WC_Pe = round((pe_ratio_list[0] + np.mean(pe_ratio_list[1:4])) / 2)
        Buy_Pe = np.mean(pe_ratio_list[1:4])
        Sell_Pe = np.mean(pe_ratio_list[-4:-1])
        FV_Pe = (Buy_Pe + Sell_Pe) / 2

        WC_Pe_Price = eps * WC_Pe
        Buy_Pe_Price = eps * Buy_Pe
        FV_Pe_Price = eps * FV_Pe
        Sell_Pe_Price = eps * Sell_Pe

        pe_values = pd.DataFrame({
            "PE Ratio": [WC_Pe, Buy_Pe, FV_Pe, Sell_Pe]
        }, index=["WC", "Buy", "FV", "Sell"])

        pe_prices = pd.DataFrame({
            "PE Ratio": [WC_Pe_Price, Buy_Pe_Price, FV_Pe_Price, Sell_Pe_Price]
        }, index=["WC", "Buy", "FV", "Sell"])

        if WC_Pe_Price < price and Sell_Pe_Price < price:
            Chance_pe = 0
            Risk_pe = price - WC_Pe_Price
            Crv_pe = 0
        elif WC_Pe_Price > price and Sell_Pe_Price > price:
            Chance_pe = Sell_Pe_Price - price
            Risk_pe = 0
            Crv_pe = float('inf')
        else:
            Chance_pe = Sell_Pe_Price - price
            Risk_pe = price - WC_Pe_Price
            Crv_pe = Chance_pe / Risk_pe if Risk_pe != 0 else float('inf')

    # ---- Combine Results ----
    df_result = pd.concat([pe_values, ev_values], axis=1)
    df_prices = pd.concat([pe_prices, ev_prices], axis=1)

    return {
        "pe_values": pe_values,
        "pe_prices": pe_prices,
        "ev_values": ev_values,
        "ev_prices": ev_prices,
        "df_result": df_result,
        "df_prices": df_prices,
        "Crv_pe": Crv_pe,
        "Crv_ev": Crv_ev,
        "Chance_ev": Chance_ev,
        "Risk_ev": Risk_ev,
        "Chance_pe": Chance_pe,
        "Risk_pe": Risk_pe
    }


## `ev_histogram()`

Creates a histogram table for historical EV/EBITDA values:

- Divides historical EV/EBITDA data into bins.
- Shows frequency of occurrences and cumulative probabilities.
- Computes probability of upside beyond each bin.
- Used for probability-adjusted CRV calculation.


In [14]:

def ev_histogram(ev_series, bins=15):
    """
    Create a histogram table for EV/EBITDA values.

    Parameters
    ----------
    ev_series : pd.Series 
        Historical EV/EBITDA values
    bins : int 
        Number of bins to divide the data into (default: 15)

    Returns
    -------
    hist_ev : pd.DataFrame 
        Histogram table with columns:
        - "Low": lower edge of the bin
        - "High": upper edge of the bin
        - "Number of Days": count of observations in the bin
        - "% of Days": percentage of total observations in the bin
        - "Cumulative (%)": cumulative percentage up to this bin
        - "Probability Up (%)": probability of EV/EBITDA being higher than this bin
    """
    counts, edges = np.histogram(ev_series, bins=bins)
    total_days = len(ev_series)

    hist_ev = pd.DataFrame({
        "Low": edges[:-1],
        "High": edges[1:],
        "Number of Days": counts
    })
    hist_ev["% of Days"] = hist_ev["Number of Days"] / total_days * 100
    hist_ev["Cumulative (%)"] = hist_ev["% of Days"].cumsum().round(2)
    hist_ev["Probability Up (%)"] = (100 - hist_ev["Cumulative (%)"]).round(2)
    return hist_ev


## `pe_histogram()`

Same as `ev_histogram()` but for historical PE Ratios:

- Divides PE data into bins.
- Calculates % of days, cumulative %, and probability of upside.
- Supports probability-adjusted CRV for PE Ratio.


In [15]:

def pe_histogram(pe_series, bins=15):
    """
    Create a histogram table for PE Ratio values.

    Parameters
    ----------
    pe_series : pd.Series
        Historical PE Ratio values
    bins : int 
        Number of bins to divide the data into (default: 15)

    Returns
    -------
    hist_pe : pd.DataFrame
        Histogram table with columns:
        - "Low": lower edge of the bin
        - "High": upper edge of the bin
        - "Number of Days": count of observations in the bin
        - "% of Days": percentage of total observations in the bin
        - "Cumulative (%)": cumulative percentage up to this bin
        - "Probability Up (%)": probability of PE Ratio being higher than this bin
    """

    counts, edges = np.histogram(pe_series, bins=bins)
    total_days = len(pe_series)

    hist_pe = pd.DataFrame({
        "Low": edges[:-1],
        "High": edges[1:],
        "Number of Days": counts
    })
    hist_pe["% of Days"] = hist_pe["Number of Days"] / total_days * 100
    hist_pe["Cumulative (%)"] = hist_pe["% of Days"].cumsum().round(2)
    hist_pe["Probability Up (%)"] = (100 - hist_pe["Cumulative (%)"]).round(2)
    return hist_pe

## `probability_adjusted_ev()`

Calculates probability-adjusted CRV based on EV/EBITDA:

- Uses historical EV/EBITDA histogram to estimate upside/downside probabilities.
- Adjusts basic CRV by weighting chance and risk with these probabilities.
- Outputs: P_up, P_down, expected monetary value, upside/downside adjusted, probability-adjusted CRV.


In [16]:

def probability_adjusted_ev(ev_series, current_ev_ebitda, price, ev_ebitda_list, ebitda, net_debt, eps, shares_outstanding):
   
    """
    Calculate probability-adjusted Risk/Reward Ratio (CRV) based on EV/EBITDA.

    Parameters
    ----------
    ev_series : pd.Series
        Historical EV/EBITDA values.
    current_ev_ebitda : float
        Current EV/EBITDA of the company.
    price : float
        Current stock price.
    ev_ebitda_list : list of float
        Historical yearly EV/EBITDA values, sorted.
    ebitda : float
        Current EBITDA value.
    net_debt : float
        Net debt (total debt minus cash).
    eps : float
        Earnings per share.
    shares_outstanding : int
        Number of shares outstanding.

    Returns
    -------
    dict 
        Contains probability-adjusted CRV calculations:
        - "P_up": probability of upside
        - "P_down": probability of downside
        - "Expected_Value": expected monetary value
        - "Upside_adjusted": upside weighted by probability
        - "Downside_adjusted": downside weighted by probability
        - "CRV_adjusted": probability-adjusted Risk/Reward Ratio
    """

    hist_ev = ev_histogram(ev_series, bins=15)

    P_down = hist_ev.loc[hist_ev["High"] <= current_ev_ebitda, "% of Days"].sum() / 100
    P_up = hist_ev.loc[hist_ev["Low"] > current_ev_ebitda, "% of Days"].sum() / 100

    crv_data = calculate_crv(price, ev_ebitda_list, None, ebitda, net_debt, eps, shares_outstanding, has_ev=True, has_pe=False)
    Chance_ev = crv_data["Chance_ev"]
    Risk_ev = crv_data["Risk_ev"]

    Expected_Value = P_up * Chance_ev - P_down * Risk_ev
    Upside_adjusted = P_up * Chance_ev
    Downside_adjusted = P_down * Risk_ev

    CRV_adjusted = Upside_adjusted / Downside_adjusted if Downside_adjusted > 0 else float('inf')

    return {
        "P_up": P_up,
        "P_down": P_down,
        "Expected_Value": Expected_Value,
        "Upside_adjusted": Upside_adjusted,
        "Downside_adjusted": Downside_adjusted,
        "CRV_adjusted": CRV_adjusted
    }

## `probability_adjusted_pe()`

Calculates probability-adjusted CRV based on PE Ratio:

- Uses historical PE histogram to estimate upside/downside probabilities.
- Adjusts basic CRV by weighting chance and risk with these probabilities.
- Outputs: P_up, P_down, expected monetary value, upside/downside adjusted, probability-adjusted CRV.


In [17]:

def probability_adjusted_pe(pe_series, current_pe_ratio, price, pe_ratio_list, ebitda, net_debt, eps, shares_outstanding):
    """
    Calculate probability-adjusted Risk/Reward Ratio (CRV) based on PE Ratio.

    Parameters
    ----------
    pe_series : pd.Series
        Historical PE Ratio values.
    current_pe_ratio : float
        Current PE Ratio of the company.
    price : float
        Current stock price.
    pe_ratio_list : list of float
        Historical yearly PE Ratio values, sorted.
    ebitda : float
        Current EBITDA value.
    net_debt : float
        Net debt (total debt minus cash).
    eps : float
        Earnings per share.
    shares_outstanding : int
        Number of shares outstanding.

    Returns
    -------
    dict
        Contains probability-adjusted CRV calculations:
        - "P_up": probability of upside
        - "P_down": probability of downside
        - "Expected_Value": expected monetary value
        - "Upside_adjusted": upside weighted by probability
        - "Downside_adjusted": downside weighted by probability
        - "CRV_adjusted": probability-adjusted Risk/Reward Ratio
    """
    
    hist_pe = pe_histogram(pe_series, bins=15)

    P_down = hist_pe.loc[hist_pe["High"] <= current_pe_ratio, "% of Days"].sum() / 100
    P_up = hist_pe.loc[hist_pe["Low"] > current_pe_ratio, "% of Days"].sum() / 100

    crv_data = calculate_crv(price, None, pe_ratio_list, ebitda, net_debt, eps, shares_outstanding, has_ev=False, has_pe=True)
    Chance_pe = crv_data["Chance_pe"]
    Risk_pe = crv_data["Risk_pe"]

    Expected_Value = P_up * Chance_pe - P_down * Risk_pe
    Upside_adjusted = P_up * Chance_pe
    Downside_adjusted = P_down * Risk_pe

    CRV_adjusted = Upside_adjusted / Downside_adjusted if Downside_adjusted > 0 else float('inf')

    return {
        "P_up": P_up,
        "P_down": P_down,
        "Expected_Value": Expected_Value,
        "Upside_adjusted": Upside_adjusted,
        "Downside_adjusted": Downside_adjusted,
        "CRV_adjusted": CRV_adjusted
    }

## Step 1: Prepare Data

We prepare historical EV/EBITDA and PE Ratio data, current EBITDA, current PE Ratio, net debt, 
EPS, and other financial data.


In [18]:
# Check which columns are available
columns = annual_avg.columns
has_pe = f"{TICKER}: PE Ratio" in columns
has_ev = f"{TICKER}: EV/EBITDA" in columns

# Calculate financial metrics
net_income = info.financials.loc["Net Income"].iloc[0]
eps = net_income / shares_outstanding
ebitda = info.financials.loc["EBITDA"].iloc[0]
Debt = balance_sheet.loc["Total Debt"].iloc[0]
cash = balance_sheet.loc["Cash And Cash Equivalents"].iloc[0]
long_term_debt = df[f"{TICKER}: Quarterly Long Term Debt"].iloc[-1]
net_debt = long_term_debt - cash

# Prepare EV/EBITDA data
if has_ev:
    ev_ebitda_list = annual_avg[f"{TICKER}: EV/EBITDA"].dropna().tolist()
    ev_ebitda_list.sort()
    ev_ebitda = float(df[f"{TICKER}: EV/EBITDA"].iloc[-1])
    ev_series = df[f"{TICKER}: EV/EBITDA"].dropna()

# Prepare PE Ratio data
if has_pe:
    pe_ratio_list = annual_avg[f"{TICKER}: PE Ratio"].dropna().tolist()
    pe_ratio_list.sort()
    pe_ratio = float(df[f"{TICKER}: PE Ratio"].iloc[-1])
    pe_series = df[f"{TICKER}: PE Ratio"].dropna()

## Step 2: Calculate CRV


In [19]:

# =========================
# Example Usage for Risk-Reward Ratio Calculation
# =========================

# Calculate CRV for both EV/EBITDA and PE
crv_data = calculate_crv(
    price=price,
    ev_ebitda_list=ev_ebitda_list if has_ev else None,
    pe_ratio_list=pe_ratio_list if has_pe else None,
    ebitda=ebitda,
    net_debt=net_debt,
    eps=eps,
    shares_outstanding=shares_outstanding,
    has_ev=has_ev,
    has_pe=has_pe
)

# Calculate histograms and probability-adjusted CRV
if has_ev:
    hist_ev = ev_histogram(ev_series, bins=15)
    chance_data_ev = probability_adjusted_ev(
        ev_series=ev_series,
        current_ev_ebitda=ev_ebitda,
        price=price,
        ev_ebitda_list=ev_ebitda_list,
        ebitda=ebitda,
        net_debt=net_debt,
        eps=eps,
        shares_outstanding=shares_outstanding
    )

if has_pe:
    hist_pe = pe_histogram(pe_series, bins=15)
    chance_data_pe = probability_adjusted_pe(
        pe_series=pe_series,
        current_pe_ratio=pe_ratio,
        price=price,
        pe_ratio_list=pe_ratio_list,
        ebitda=ebitda,
        net_debt=net_debt,
        eps=eps,
        shares_outstanding=shares_outstanding
    )



## Step 3: Display Results

In [20]:

# =========================
# Print Results
# =========================

print("- Risk-Reward Analysis -\n")
print(f"current Price: ${price:,.2f}")
if has_ev:
    print(f"EV/EBITDA: {ev_ebitda:,.2f}")
if has_pe: 
    print(f"PE Ratio: {pe_ratio:,.2f}")

print("\nPrices:\n")
display(crv_data["df_prices"])
print("\nKennzahlen:\n")
display(crv_data["df_result"])
print("\nHistogram Table for EV/EBITDA:\n")
if has_ev:
    display(hist_ev)
print("\nHistogram Table for PE Ratio:\n")
if has_pe:
    display(hist_pe)
print("\n")

if has_ev:
    print("- EV/EBITDA Risk-Reward Ratio -\n")
    print(f"Risk-Reward Ratio: {crv_data['Crv_ev']} : 1 \n")
    print(f"P(Up): {chance_data_ev['P_up']:.2%}")
    print(f"P(Down): {chance_data_ev['P_down']:.2%}")
    print(f"Chance: {crv_data['Chance_ev']:.2f}")
    print(f"Risk: {crv_data['Risk_ev']:.2f}")
    print(f'Probability-Adjusted Expected Value: {chance_data_ev["Expected_Value"]:.2f}')
    print(f"Upside Adjusted: {chance_data_ev['Upside_adjusted']:.2f}")
    print(f"Downside Adjusted: {chance_data_ev['Downside_adjusted']:.2f}")
    print(f'Probability-Adjusted Risk-Reward Ratio: {chance_data_ev["CRV_adjusted"]:.2f} : 1 \n')

if has_pe:
    print("- PE Ratio Risk-Reward Ratio -\n")
    print(f"Risk-Reward Ratio: {crv_data['Crv_pe']} : 1 \n")
    print(f"P(Up): {chance_data_pe['P_up']:.2%}")
    print(f"P(Down): {chance_data_pe['P_down']:.2%}")
    print(f"Chance: {crv_data['Chance_pe']:.2f}")
    print(f"Risk: {crv_data['Risk_pe']:.2f}")
    print(f'Probability-Adjusted Expected Value: {chance_data_pe["Expected_Value"]:.2f}')
    print(f"Upside Adjusted: {chance_data_pe['Upside_adjusted']:.2f}")
    print(f"Downside Adjusted: {chance_data_pe['Downside_adjusted']:.2f}")
    print(f'Probability-Adjusted Risk-Reward Ratio: {chance_data_pe["CRV_adjusted"]:.2f} : 1 \n')



- Risk-Reward Analysis -

current Price: $449.23
EV/EBITDA: 20.73
PE Ratio: 34.70

Prices:



,PE Ratio,EV/EBITDA
WC,164.413603,140.960813
Buy,184.537498,143.429350
FV,335.184617,314.006425
Sell,485.831735,484.583499



Kennzahlen:



,PE Ratio,EV/EBITDA
WC,12.000000,7.000000
Buy,13.468776,7.114551
FV,24.464006,15.030096
Sell,35.459236,22.945641



Histogram Table for EV/EBITDA:



,Low,High,Number of Days,% of Days,Cumulative (%),Probability Up (%)
0,4.520000,6.163333,75,7.197697,7.20,92.80
1,6.163333,7.806667,149,14.299424,21.50,78.50
2,7.806667,9.450000,125,11.996161,33.49,66.51
3,9.450000,11.093333,60,5.758157,39.25,60.75
4,11.093333,12.736667,76,7.293666,46.55,53.45
5,12.736667,14.380000,95,9.117083,55.66,44.34
6,14.380000,16.023333,65,6.238004,61.90,38.10
7,16.023333,17.666667,55,5.278311,67.18,32.82
8,17.666667,19.310000,61,5.854127,73.03,26.97
9,19.310000,20.953333,40,3.838772,76.87,23.13



Histogram Table for PE Ratio:



,Low,High,Number of Days,% of Days,Cumulative (%),Probability Up (%)
0,8.130,11.266,102,8.710504,8.71,91.29
1,11.266,14.402,120,10.247652,18.96,81.04
2,14.402,17.538,163,13.919727,32.88,67.12
3,17.538,20.674,86,7.344150,40.22,59.78
4,20.674,23.810,94,8.027327,48.25,51.75
5,23.810,26.946,100,8.539710,56.79,43.21
6,26.946,30.082,113,9.649872,66.44,33.56
7,30.082,33.218,77,6.575576,73.01,26.99
8,33.218,36.354,158,13.492741,86.51,13.49
9,36.354,39.490,91,7.771136,94.28,5.72




- EV/EBITDA Risk-Reward Ratio -

Risk-Reward Ratio: 0.11468385703826497 : 1 

P(Up): 23.13%
P(Down): 73.03%
Chance: 35.35
Risk: 308.27
Probability-Adjusted Expected Value: -216.96
Upside Adjusted: 8.18
Downside Adjusted: 225.14
Probability-Adjusted Risk-Reward Ratio: 0.04 : 1 

- PE Ratio Risk-Reward Ratio -

Risk-Reward Ratio: 0.1285099305321626 : 1 

P(Up): 13.49%
P(Down): 73.01%
Chance: 36.60
Risk: 284.82
Probability-Adjusted Expected Value: -203.02
Upside Adjusted: 4.94
Downside Adjusted: 207.96
Probability-Adjusted Risk-Reward Ratio: 0.02 : 1 

